In [2]:
!pip install h5py


In [20]:
!python -m pip install --upgrade pip



  Using cached pip-23.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.2.3
    Uninstalling pip-20.2.3:
      Successfully uninstalled pip-20.2.3


In [22]:
!pip install grpcio>=1.48.2


In [21]:
!pip install tensorflow

  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached grpcio-1.54.0-cp38-cp38-win_amd64.whl (4.1 MB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.47.0
    Uninstalling grpcio-1.47.0:
      Successfully uninstalled grpcio-1.47.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.47.0 requires protobuf<4.0dev,>=3.12.0, but you have protobuf 4.22.3 which is incompatible.


In [5]:
import gzip
import shutil

input_file = 'camelyonpatch_level_2_split_test_x.h5.gz'
output_file = 'camelyonpatch_level_2_split_test_x.h5'

with gzip.open(input_file, 'rb') as f_in:
    with open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [6]:
import gzip
import shutil

input_file = 'camelyonpatch_level_2_split_test_y.h5.gz'
output_file = 'camelyonpatch_level_2_split_test_y.h5'

with gzip.open(input_file, 'rb') as f_in:
    with open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [19]:
import h5py
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import h5py

def split_data(x, y, train_ratio, val_ratio, test_ratio):
    assert train_ratio + val_ratio + test_ratio == 1, "The sum of the ratios should be equal to 1"

    total_samples = len(x)
    train_size = int(total_samples * train_ratio)
    val_size = int(total_samples * val_ratio)

    indices = np.arange(total_samples)
    np.random.shuffle(indices)

    train_indices = indices[:train_size]
    val_indices = indices[train_size:train_size + val_size]
    test_indices = indices[train_size + val_size:]

    x_train, y_train = x[train_indices], y[train_indices]
    x_val, y_val = x[val_indices], y[val_indices]
    x_test, y_test = x[test_indices], y[test_indices]

    return (x_train, y_train), (x_val, y_val), (x_test, y_test)

with h5py.File('camelyonpatch_level_2_split_test_x.h5', 'r') as f:
    x_data = f['x'][:]

with h5py.File('camelyonpatch_level_2_split_test_y.h5', 'r') as f:
    y_data = f['y'][:]

(x_train, y_train), _ , (x_test, y_test) = split_data(x_data, y_data, 0.2, 0.6, 0.2)

datagen = ImageDataGenerator(
    preprocessing_function=lambda x: x / 255.,
    width_shift_range=4,  # randomly shift images horizontally
    height_shift_range=4,  # randomly shift images vertically
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images


In [20]:
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)


x_train shape: (6553, 96, 96, 3)
y_train shape: (6553, 1, 1, 1)


In [21]:
import numpy as np

y_test = np.reshape(y_test, (y_test.shape[0], 1))
y_train = np.reshape(y_train, (y_train.shape[0], 1))

In [45]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

# Define your model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(30, activation='relu'))  # Added a new 30-neuron dense layer
model.add(Dense(1, activation='sigmoid'))


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit_generator(datagen.flow(x_train, y_train, batch_size=256),
                    steps_per_epoch=len(x_train) // 256,
                    epochs=10)


C:\Users\HP\AppData\Local\Temp\ipykernel_22208\2437059189.py:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train, y_train, batch_size=256),


Epoch 1/10
25/25 [==============================] - 22s 832ms/step - loss: 0.6765 - accuracy: 0.6400
Epoch 2/10
25/25 [==============================] - 20s 809ms/step - loss: 0.4946 - accuracy: 0.7732
Epoch 3/10
25/25 [==============================] - 21s 815ms/step - loss: 0.4811 - accuracy: 0.7778
Epoch 4/10
25/25 [==============================] - 21s 824ms/step - loss: 0.4687 - accuracy: 0.7820
Epoch 5/10
25/25 [==============================] - 20s 810ms/step - loss: 0.4397 - accuracy: 0.7994
Epoch 6/10
25/25 [==============================] - 21s 827ms/step - loss: 0.4320 - accuracy: 0.8023
Epoch 7/10
25/25 [==============================] - 21s 821ms/step - loss: 0.4260 - accuracy: 0.8009
Epoch 8/10
25/25 [==============================] - 21s 824ms/step - loss: 0.4395 - accuracy: 0.7959
Epoch 9/10
25/25 [==============================] - 21s 818ms/step - loss: 0.4277 - accuracy: 0.8059
Epoch 10/10
25/25 [==============================] - 22s 870ms/step - loss: 0.4361 - accura

In [46]:
y_train_binary = y_train.flatten()
model_preds = np.round(model.predict(x_train)).flatten()
accuracy = accuracy_score(y_train_binary, model_preds)
f1 = f1_score(y_train_binary, model_preds)
print("Comparison between model and y_train")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

205/205 [==============================] - 5s 27ms/step
Comparison between model and y_train
Accuracy: 0.7966
F1 Score: 0.7937


In [75]:
incorrect_preds.sum()

1333

In [67]:
anomaly_labels.sum()

1311

In [87]:

# 2. Train Isolation Forest, Local Outlier Factor, and One-Class SVM models on the extracted features
iso_forest = IsolationForest(contamination=0.2)
lof = LocalOutlierFactor(contamination=0.2, novelty=True)
one_class_svm = OneClassSVM(nu=0.1)

iso_forest.fit(x_train_features)
lof.fit(x_train_features)
one_class_svm.fit(x_train_features)

# 3. Make predictions using the three models
iso_forest_pred = iso_forest.predict(x_train_features)
lof_pred = lof.predict(x_train_features)
one_class_svm_pred = one_class_svm.predict(x_train_features)

# Combine the results using a voting scheme
combined_pred = mode(np.vstack((iso_forest_pred, lof_pred, one_class_svm_pred)), axis=0)[0].flatten()

# 4. Evaluate the models and the combined results
models = [iso_forest_pred, lof_pred, one_class_svm_pred, combined_pred]
model_names = ['Isolation Forest', 'Local Outlier Factor', 'One-Class SVM', 'Combined (Voting)']

model_preds = np.round(model.predict(x_train)).flatten()
   
for name, pred in zip(model_names, models):
    print(f'\n{name} Results:')

    # Convert anomalies to binary labels (1 for normal, -1 for anomaly)
    binary_pred = (pred == 1).astype(int)
    binary_y_train = (y_train.flatten() == 0).astype(int)

    # Compare with incorrect model predictions
    incorrect_pred = model_preds != binary_y_train

    TP = np.sum((incorrect_pred == 1) & (binary_pred == 1))
    FP = np.sum((incorrect_pred == 0) & (binary_pred == 1))
    FN = np.sum((incorrect_pred == 1) & (binary_pred == 0))

    if TP + FN != 0:
        recall = TP / (TP + FN)
    else:
        recall = 0

    if TP + FP != 0:
        precision = TP / (TP + FP)
    else:
        precision = 0

    print('Comparison between incorrect model predictions and detected anomalies')
  
    print('Recall (Wrong labels detected / All wrong labels):', recall)
    print('Precision (Wrong labels detected / All anomalies detected):', precision)



205/205 [==============================] - 5s 23ms/step

Isolation Forest Results:
Comparison between incorrect model predictions and detected anomalies
Recall (Wrong labels detected / All wrong labels): 0.7862068965517242
Precision (Wrong labels detected / All anomalies detected): 0.7829072872949256

Local Outlier Factor Results:
Comparison between incorrect model predictions and detected anomalies
Recall (Wrong labels detected / All wrong labels): 0.8176245210727969
Precision (Wrong labels detected / All anomalies detected): 0.7934560327198364

One-Class SVM Results:
Comparison between incorrect model predictions and detected anomalies
Recall (Wrong labels detected / All wrong labels): 0.882183908045977
Precision (Wrong labels detected / All anomalies detected): 0.7807731434384537

Combined (Voting) Results:
Comparison between incorrect model predictions and detected anomalies
Recall (Wrong labels detected / All wrong labels): 0.8683908045977011
Precision (Wrong labels detected / All

In [89]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model

input_shape = (96, 96, 3)

# Encoder
input_img = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Autoencoder model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=256,
                shuffle=True,
                validation_split=0.2)

# Encoder model to extract features
encoder = Model(input_img, encoded)


Epoch 1/10
21/21 [==============================] - 52s 2s/step - loss: 30107.1016 - val_loss: 30614.9316
Epoch 2/10
21/21 [==============================] - 51s 2s/step - loss: 30099.9609 - val_loss: 30614.9316
Epoch 3/10
21/21 [==============================] - 52s 2s/step - loss: 30099.9590 - val_loss: 30614.9316
Epoch 4/10
21/21 [==============================] - 52s 2s/step - loss: 30099.9551 - val_loss: 30614.9316
Epoch 5/10
21/21 [==============================] - 52s 2s/step - loss: 30099.9590 - val_loss: 30614.9316
Epoch 6/10
21/21 [==============================] - 52s 2s/step - loss: 30099.9590 - val_loss: 30614.9316
Epoch 7/10
21/21 [==============================] - 52s 2s/step - loss: 30099.9609 - val_loss: 30614.9316
Epoch 8/10
21/21 [==============================] - 52s 2s/step - loss: 30099.9609 - val_loss: 30614.9316
Epoch 9/10
21/21 [==============================] - 53s 3s/step - loss: 30099.9551 - val_loss: 30614.9316
Epoch 10/10
21/21 [===========================

In [90]:
# Encode the train and test datasets
x_train_encoded = encoder.predict(x_train)
x_test_encoded = encoder.predict(x_test)

# Flatten the encoded representations
x_train_encoded = x_train_encoded.reshape(x_train_encoded.shape[0], -1)
x_test_encoded = x_test_encoded.reshape(x_test_encoded.shape[0], -1)

# Train the Isolation Forest on encoded features
iso_forest_encoded = IsolationForest(contamination=0.1)
iso_forest_encoded.fit(x_train_encoded)

# Train the Local Outlier Factor on encoded features
lof_encoded = LocalOutlierFactor(contamination=0.1)
lof_encoded.fit(x_train_encoded)

# Train the One-Class SVM on encoded features
oc_svm_encoded = OneClassSVM(nu=0.1, kernel='rbf', gamma='scale')
oc_svm_encoded.fit(x_train_encoded)

# Define a function to compare results
def compare_results(test_data, test_labels, anomaly_model):
    pred = anomaly_model.predict(test_data)
    binary_pred = (pred == 1).astype(int)
    binary_labels = (test_labels == 0).astype(int).flatten()
    incorrect_pred = (model.predict(test_data).round().flatten() != binary_labels).astype(int)

    accuracy = accuracy_score(binary_labels, binary_pred)
    f1 = f1_score(binary_labels, binary_pred)
    detection_rate = np.sum(incorrect_pred * binary_pred) / np.sum(incorrect_pred)
    false_positive_rate = np.sum(incorrect_pred * binary_pred) / np.sum(binary_pred)
    
    return accuracy, f1, detection_rate, false_positive_rate

# Compare results for each anomaly detection algorithm
for name, anomaly_model in [('Isolation Forest', iso_forest_encoded),
                            ('Local Outlier Factor', lof_encoded),
                            ('One-Class SVM', oc_svm_encoded)]:
    accuracy, f1, detection_rate, false_positive_rate = compare_results(x_test_encoded, y_test, anomaly_model)
    print(f"{name}\nAccuracy: {accuracy:.4f}\nF1 Score: {f1:.4f}\nDetection Rate: {detection_rate:.4f}\nFalse Positive Rate: {false_positive_rate:.4f}\n")


205/205 [==============================] - 5s 25ms/step


KeyboardInterrupt: 